In [ ]:
%matplotlib ipympl

import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import xarray as xr
import pandas as pd
import numpy as np

from global_settings import cosmo_path, start_date
from domain_definition import get_develop_domain

outer_dom, inner_dom = get_develop_domain()

lon_min = outer_dom.proj.lon.min() - 0.05
lon_max = outer_dom.proj.lon.max() + 0.05

lat_min = outer_dom.proj.lat.min() - 0.05
lat_max = outer_dom.proj.lat.max() + 0.05

In [ ]:
# COSMO data, pre-processed.
cosmo_ds = xr.open_dataset(f'{cosmo_path}/COSMO_CTRL_BC_nD_LES.nc')
cosmo_ds = cosmo_ds.sel(rlat=slice(lat_min, lat_max), rlon=slice(lon_min, lon_max))

In [ ]:
# MicroHH cross-sections.
outer_path = 'develop_case/outer'
inner_path = 'develop_case/inner'

def read(filename):
    """Read a file and return the data."""
    ds = xr.open_dataset(filename, decode_times=False)
    ds['time'] = pd.to_datetime(ds['time'].values.astype(np.int32), unit='s', origin=start_date)
    return ds

thl_out = read(f'{outer_path}/thl.xy.nc')
qt_out  = read(f'{outer_path}/qt.xy.nc')
u_out   = read(f'{outer_path}/u.xy.nc')
v_out   = read(f'{outer_path}/v.xy.nc')

ds_out = xr.merge([thl_out, qt_out, u_out, v_out])

thl_inn = read(f'{inner_path}/thl.xy.nc')
qt_inn  = read(f'{inner_path}/qt.xy.nc')
u_inn   = read(f'{inner_path}/u.xy.nc')
v_inn   = read(f'{inner_path}/v.xy.nc')

ds_inn = xr.merge([thl_inn, qt_inn, u_inn, v_inn])

In [ ]:
# Plot!
date = start_date + timedelta(hours=1)

ds1 = cosmo_ds.sel(time=date, method='nearest')
ds2 = ds_out.sel(time=date, method='nearest')
ds3 = ds_inn.sel(time=date, method='nearest')

var = 'qt'

vmin = ds1[var][0,:,:].min().values
vmax = ds1[var][0,:,:].max().values

plt.figure(figsize=(10,6), layout='constrained')
plt.pcolormesh(ds1.rlon, ds1.rlat, ds1[var][0,:,:], vmin=vmin, vmax=vmax, cmap=plt.cm.RdBu_r)
plt.pcolormesh(outer_dom.proj.lon, outer_dom.proj.lat, ds2[var][0], vmin=vmin, vmax=vmax, cmap=plt.cm.RdBu_r)
plt.pcolormesh(inner_dom.proj.lon, inner_dom.proj.lat, ds3[var][0], vmin=vmin, vmax=vmax, cmap=plt.cm.RdBu_r)
plt.plot(outer_dom.proj.bbox_lon, outer_dom.proj.bbox_lat, 'k:')
plt.plot(inner_dom.proj.bbox_lon, inner_dom.proj.bbox_lat, 'k:')
plt.colorbar()